# importing libraries

In [2]:
import requests 
import re
import os
from bs4 import BeautifulSoup
import assemblyai as aai

In [3]:
# the xml content
rss_feed_url = "https://feed.firstory.me/rss/user/cjzdxzf7vbzvj075872kanoib"
page = requests.get(rss_feed_url)
soup = BeautifulSoup(page.content,'xml')

# creating a key in OS which have the api

In [22]:
key = os.environ['ASSEMPLY_KEY']

In [24]:
key[:3]

'401'

# Downloading Podcasts

In [5]:
# os.mkdir('./downloads')
podcast_items = soup.find_all('item')
count = 0
for podcast in podcast_items:
    if count == 5:
        break
        
    title = podcast.find('title').text
    description = podcast.find('description').text
    mp3_url = podcast.find('enclosure')['url']
    
    if re.search(r'ai',description,re.I):
        mp3_file = requests.get(mp3_url)
        # Remove invalid characters from the title using regex
        title_cleaned = re.sub(r'[^\w]', '_', title)
        # Construct the filename using the cleaned title
        filename = f'./downloads/{title_cleaned}.mp3'
        with open(filename, 'wb') as f:
            f.write(mp3_file.content)

        count += 1


# Transcribing Podcasts

In [21]:
import os
import assemblyai as aai

# Replace with your AssemblyAI API key
key = os.environ['ASSEMPLY_KEY']
aai.settings.api_key = key

# Path to the downloads folder
downloads_folder = './downloads'

# Initialize the transcriber outside the loop
transcriber = aai.Transcriber()

# Iterate over all files in the downloads folder
for filename in os.listdir(downloads_folder):
    file_path = os.path.join(downloads_folder, filename)

    # Check if the item is a file (not a directory)
    if os.path.isfile(file_path):
        try:
            # Transcribe the file
            transcript = transcriber.transcribe(file_path)

            # Create a folder for transcriptions
            transcriptions_folder = './transcriptions'
            os.makedirs(transcriptions_folder, exist_ok=True)

            # Save the transcription to a text file
            file_name = os.path.splitext(filename)[0]
            with open(f'{transcriptions_folder}/{file_name}_transcription.txt', 'w') as f:
                f.write(transcript.text)
            
        except aai.exceptions.BadRequestError as e:
            print(f"Error transcribing {filename}: {e}")
